In [2]:
import pandas as pd
import numpy as np
import pickle as pk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
import re
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import load_model
import tensorflow




def trainIntentModel():
    # Load the dataset and prepare it to the train the model

    # Importing dataset and splitting into words and labels
    dataset = pd.read_csv('datasets/intent.csv', names=["Query", "Intent"])

    X = dataset["Query"]
    y = dataset["Intent"]

    unique_intent_list = list(set(y))

    print("Intent Dataset successfully loaded!")
    
    # Clean and prepare the intents corpus
    queryCorpus = []
    ps = PorterStemmer()

    for query in X:
        query = re.sub('[^a-zA-Z]', ' ', query)

        # Tokenize sentence
        query = query.split(' ')

        # Lemmatizing
        tokenized_query = [ps.stem(word.lower()) for word in query]

        # Recreate the sentence from tokens
        tokenized_query = ' '.join(tokenized_query)

        # Add to corpus
        queryCorpus.append(tokenized_query)
        
    print(queryCorpus)
    print("Corpus created")
    
    countVectorizer= CountVectorizer(max_features=800)
    corpus = countVectorizer.fit_transform(queryCorpus).toarray()
    print(corpus.shape)
    print("Bag of words created!")
    
    # Save the CountVectorizer
    pk.dump(countVectorizer, open("saved_state/IntentCountVectorizer.sav", 'wb'))
    print("Intent CountVectorizer saved!")
    
    # Encode the intent classes
    labelencoder_intent = LabelEncoder()
    y = labelencoder_intent.fit_transform(y)
    y = np_utils.to_categorical(y)
    print("Encoded the intent classes!")
    print(y)
    
    # Return a dictionary, mapping labels to their integer values
    res = {}
    for cl in labelencoder_intent.classes_:
        res.update({cl:labelencoder_intent.transform([cl])[0]})

    intent_label_map = res
    print(intent_label_map)
    print("Intent Label mapping obtained!")
    
    # Initialising the Aritifcial Neural Network
    classifier = Sequential()

    # Adding the input layer and the first hidden layer
    classifier.add(Dense(units = 96, kernel_initializer = 'uniform', activation = 'relu', input_dim = 133))

    # Adding the second hidden layer
    classifier.add(Dense(units = 96, kernel_initializer = 'uniform', activation = 'relu'))

    # Adding the output layer
    classifier.add(Dense(units = 35, kernel_initializer = 'uniform', activation = 'softmax'))

    # Compiling the ANN
    classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    # Fitting the ANN to the Training set
    classifier.fit(corpus, y, batch_size = 10, epochs = 500)
    
    return classifier, intent_label_map






intent_model, intent_label_map = trainIntentModel()

# Save the Intent model
intent_model.save('saved_state/intent_model.h5')
print("Intent model saved!")



def trainEntityModel():
    # Importing dataset and splitting into words and labels
    dataset = pd.read_csv('datasets/data-tags.csv', names=["word", "label"])
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, 1].values
#     X = X.reshape(630,)
    print(X)
    print("Entity Dataset successfully loaded!")

    entityCorpus=[]
    ps = PorterStemmer()

    # Stem words in X
    for word in X.astype(str):
        word = [ps.stem(word[0])]
        entityCorpus.append(word)
    
    print(entityCorpus)
    X = entityCorpus
    from numpy import array
    X = array(X)
    X = X.reshape(len(X),)
    
    # Create a bag of words model for words
    from sklearn.feature_extraction.text import CountVectorizer
    cv = CountVectorizer(max_features=1500)
#     X = cv.fit_transform(X.astype('U')).toarray()
    X = cv.fit_transform(X).toarray()
    print("Entity Bag of words created!")
    
    # Save CountVectorizer state
    pk.dump(cv, open('saved_state/EntityCountVectorizer.sav', 'wb'))
    print("Entity CountVectorizer state saved!")
    
    # Encoding categorical data of labels
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y.astype(str))
    print("Encoded the entity classes!")
    
    # Return a dict mapping labels to their integer values
    res = {}
    for cl in labelencoder_y.classes_:
        res.update({cl:labelencoder_y.transform([cl])[0]})
    entity_label_map = res
    print("Entity Label mapping obtained!")
    
    # Fit classifier to dataset
    classifier = GaussianNB()
    classifier.fit(X, y)
    print("Entity Model trained successfully!")
    
    # Save the entity classifier model
    pk.dump(classifier, open('saved_state/entity_model.sav', 'wb'))
    print("Trained entity model saved!")
    
    return entity_label_map




# Load Entity model
entity_label_map = trainEntityModel()

loadedEntityCV = pk.load(open('saved_state/EntityCountVectorizer.sav', 'rb'))
loadedEntityClassifier = pk.load(open('saved_state/entity_model.sav', 'rb'))




def getEntities(query):
    query = loadedEntityCV.transform(query).toarray()
    
    response_tags = loadedEntityClassifier.predict(query)
    
    entity_list=[]
    for tag in response_tags:
        if tag in entity_label_map.values():
            entity_list.append(list(entity_label_map.keys())[list(entity_label_map.values()).index(tag)])

    return entity_list





import json
import random

with open('datasets/intents.json') as json_data:
    intents = json.load(json_data)

# Load model to predict user result
loadedIntentClassifier = load_model('saved_state/intent_model.h5')
loaded_intent_CV = pk.load(open('saved_state/IntentCountVectorizer.sav', 'rb'))    

USER_INTENT = ""


def chatbot_response(msg):
    query = re.sub('[^a-zA-Z]', ' ', msg)
    # Tokenize sentence
    query = query.split(' ')

    # Lemmatizing
    ps = PorterStemmer()
    tokenized_query = [ps.stem(word.lower()) for word in query]

    # Recreate the sentence from tokens
    processed_text = ' '.join(tokenized_query)

    # Transform the query using the CountVectorizer
    processed_text = loaded_intent_CV.transform([processed_text]).toarray()

    # Make the prediction
    predicted_Intent = loadedIntentClassifier.predict(processed_text)
#     print(predicted_Intent)
    result = np.argmax(predicted_Intent, axis=1)

    for key, value in intent_label_map.items():
        if value==result[0]:
            #print(key)
            USER_INTENT = key
            break

    for i in intents['intents']:
        if i['tag'] == USER_INTENT:
            return random.choice(i['responses'])


    # Extract entities from text
    entities = getEntities(tokenized_query)

    # Mapping between tokens and entity tags
    token_entity_map = dict(zip(entities, tokenized_query))
#     print(token_entity_map)


from tkinter import *

def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
base = Tk()
base.title("Agriculture Chatbot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)
ChatLog.place(x=6,y=6, height=386, width=370)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
scrollbar.place(x=376,y=6, height=386)

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',command= send)
SendButton.place(x=6, y=401, height=90)

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
EntryBox.place(x=128, y=401, height=90, width=265)

base.mainloop()


Intent Dataset successfully loaded!
['queri', 'hello bot ', 'hi', 'hello ', 'hey there', 'hey', 'namaskar', 'namast', 'bye', 'see you later', 'goodby', 'thank', 'thank you', 'that s help', 'thank  you so much', 'what hour are you open ', 'what are your hour ', 'when are you open ', 'how i can buy crop onlin ', 'how i can buy crop from farmer ', 'how can i buy a product ', 'how i can sell crop onlin ', 'how i can sell my crop ', 'best way to sell product ', 'how farmer can sell their product ', 'are you open today ', 'when do you open today ', 'what are your hour today ', 'who is your father ', 'what is your father s name ', 'what is your mother s name ', 'do you have parent ', 'do you have sibl ', 'who is you wife ', 'what is your wife s name ', 'are you marri ', 'are you a god ', 'how old are you ', 'are you a man ', 'are you a bot ', 'do you have a job ', 'who is your best friend ', 'am i your friend ', 'how are you ', 'how you do ', 'how are you do ', 'are you do well ', 'are you ok

22/22 [==============================] - 0s 1ms/step - loss: 1.1489 - accuracy: 0.8385
Epoch 13/500
22/22 [==============================] - 0s 1ms/step - loss: 1.0496 - accuracy: 0.8102
Epoch 14/500
22/22 [==============================] - 0s 950us/step - loss: 0.7200 - accuracy: 0.8950
Epoch 15/500
22/22 [==============================] - 0s 950us/step - loss: 0.6055 - accuracy: 0.8882
Epoch 16/500
22/22 [==============================] - 0s 950us/step - loss: 0.5464 - accuracy: 0.9227
Epoch 17/500
22/22 [==============================] - 0s 855us/step - loss: 0.3906 - accuracy: 0.9416
Epoch 18/500
22/22 [==============================] - 0s 976us/step - loss: 0.3246 - accuracy: 0.9558
Epoch 19/500
22/22 [==============================] - 0s 950us/step - loss: 0.2908 - accuracy: 0.9554
Epoch 20/500
22/22 [==============================] - 0s 1ms/step - loss: 0.1935 - accuracy: 0.9616
Epoch 21/500
22/22 [==============================] - 0s 934us/step - loss: 0.1933 - accuracy: 0.9602

Epoch 93/500
22/22 [==============================] - 0s 1ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 94/500
22/22 [==============================] - 0s 1ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 95/500
22/22 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 96/500
22/22 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 97/500
22/22 [==============================] - 0s 1ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 98/500
22/22 [==============================] - 0s 1ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 99/500
22/22 [==============================] - 0s 1ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 100/500
22/22 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 101/500
22/22 [==============================] - 0s 1ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 102/500
22/22 [==============================] - 0s 1ms/step - loss: 0.0013 - accuracy: 1.00

22/22 [==============================] - 0s 1ms/step - loss: 2.5556e-04 - accuracy: 1.0000
Epoch 172/500
22/22 [==============================] - 0s 1ms/step - loss: 3.0985e-04 - accuracy: 1.0000
Epoch 173/500
22/22 [==============================] - 0s 1ms/step - loss: 2.6185e-04 - accuracy: 1.0000
Epoch 174/500
22/22 [==============================] - 0s 1ms/step - loss: 2.8055e-04 - accuracy: 1.0000
Epoch 175/500
22/22 [==============================] - 0s 1ms/step - loss: 2.7444e-04 - accuracy: 1.0000
Epoch 176/500
22/22 [==============================] - 0s 1ms/step - loss: 2.8491e-04 - accuracy: 1.0000
Epoch 177/500
22/22 [==============================] - 0s 1ms/step - loss: 2.8617e-04 - accuracy: 1.0000
Epoch 178/500
22/22 [==============================] - 0s 1ms/step - loss: 2.6127e-04 - accuracy: 1.0000
Epoch 179/500
22/22 [==============================] - 0s 1ms/step - loss: 2.5407e-04 - accuracy: 1.0000
Epoch 180/500
22/22 [==============================] - 0s 1ms/step - 

22/22 [==============================] - 0s 1ms/step - loss: 8.7783e-05 - accuracy: 1.0000
Epoch 250/500
22/22 [==============================] - 0s 1ms/step - loss: 7.5722e-05 - accuracy: 1.0000
Epoch 251/500
22/22 [==============================] - 0s 1ms/step - loss: 9.5034e-05 - accuracy: 1.0000
Epoch 252/500
22/22 [==============================] - 0s 1ms/step - loss: 8.8361e-05 - accuracy: 1.0000
Epoch 253/500
22/22 [==============================] - 0s 1ms/step - loss: 8.7132e-05 - accuracy: 1.0000
Epoch 254/500
22/22 [==============================] - 0s 1ms/step - loss: 7.8022e-05 - accuracy: 1.0000
Epoch 255/500
22/22 [==============================] - 0s 1ms/step - loss: 7.8018e-05 - accuracy: 1.0000
Epoch 256/500
22/22 [==============================] - 0s 1ms/step - loss: 7.5947e-05 - accuracy: 1.0000
Epoch 257/500
22/22 [==============================] - 0s 1ms/step - loss: 8.5582e-05 - accuracy: 1.0000
Epoch 258/500
22/22 [==============================] - 0s 1ms/step - 

22/22 [==============================] - 0s 1ms/step - loss: 2.8790e-05 - accuracy: 1.0000
Epoch 328/500
22/22 [==============================] - 0s 1ms/step - loss: 2.7503e-05 - accuracy: 1.0000
Epoch 329/500
22/22 [==============================] - 0s 1ms/step - loss: 3.0203e-05 - accuracy: 1.0000
Epoch 330/500
22/22 [==============================] - 0s 1ms/step - loss: 3.0302e-05 - accuracy: 1.0000
Epoch 331/500
22/22 [==============================] - 0s 1ms/step - loss: 2.6739e-05 - accuracy: 1.0000
Epoch 332/500
22/22 [==============================] - 0s 1ms/step - loss: 2.8673e-05 - accuracy: 1.0000
Epoch 333/500
22/22 [==============================] - 0s 1ms/step - loss: 2.9298e-05 - accuracy: 1.0000
Epoch 334/500
22/22 [==============================] - 0s 1ms/step - loss: 2.5529e-05 - accuracy: 1.0000
Epoch 335/500
22/22 [==============================] - 0s 1ms/step - loss: 2.8423e-05 - accuracy: 1.0000
Epoch 336/500
22/22 [==============================] - 0s 1ms/step - 

22/22 [==============================] - 0s 1ms/step - loss: 1.0846e-05 - accuracy: 1.0000
Epoch 406/500
22/22 [==============================] - 0s 1ms/step - loss: 1.1039e-05 - accuracy: 1.0000
Epoch 407/500
22/22 [==============================] - 0s 1ms/step - loss: 1.0043e-05 - accuracy: 1.0000
Epoch 408/500
22/22 [==============================] - 0s 1ms/step - loss: 1.1240e-05 - accuracy: 1.0000
Epoch 409/500
22/22 [==============================] - 0s 1ms/step - loss: 1.0060e-05 - accuracy: 1.0000
Epoch 410/500
22/22 [==============================] - 0s 1ms/step - loss: 1.0035e-05 - accuracy: 1.0000
Epoch 411/500
22/22 [==============================] - 0s 1ms/step - loss: 1.0646e-05 - accuracy: 1.0000
Epoch 412/500
22/22 [==============================] - 0s 1ms/step - loss: 1.0155e-05 - accuracy: 1.0000
Epoch 413/500
22/22 [==============================] - 0s 1ms/step - loss: 9.3197e-06 - accuracy: 1.0000
Epoch 414/500
22/22 [==============================] - 0s 1ms/step - 

22/22 [==============================] - 0s 1ms/step - loss: 4.5088e-06 - accuracy: 1.0000
Epoch 484/500
22/22 [==============================] - 0s 1ms/step - loss: 4.4119e-06 - accuracy: 1.0000
Epoch 485/500
22/22 [==============================] - 0s 1ms/step - loss: 3.9566e-06 - accuracy: 1.0000
Epoch 486/500
22/22 [==============================] - 0s 1ms/step - loss: 3.9521e-06 - accuracy: 1.0000
Epoch 487/500
22/22 [==============================] - 0s 1ms/step - loss: 3.8470e-06 - accuracy: 1.0000
Epoch 488/500
22/22 [==============================] - 0s 1ms/step - loss: 3.6801e-06 - accuracy: 1.0000
Epoch 489/500
22/22 [==============================] - 0s 1ms/step - loss: 3.9562e-06 - accuracy: 1.0000
Epoch 490/500
22/22 [==============================] - 0s 1ms/step - loss: 3.5114e-06 - accuracy: 1.0000
Epoch 491/500
22/22 [==============================] - 0s 1ms/step - loss: 3.7129e-06 - accuracy: 1.0000
Epoch 492/500
22/22 [==============================] - 0s 1ms/step - 

[['word'], ['how'], ['to'], ['grow'], ['rice'], ['?'], ['jowar'], ['how'], ['much'], ['water'], ['to'], ['add'], ['for'], ['wheat'], ['?'], ['i'], ['best'], ['season'], ['to'], ['grow'], ['sugarcan'], ['!'], ['what'], ['is'], ['the'], ['yield'], ['of'], ['rice'], ['in'], ['belgaum'], ['?'], ['best'], ['pesticid'], ['for'], ['cotton'], ['.'], ['best'], ['soil'], ['for'], ['cotton'], ['.'], ['how'], ['much'], ['fertil'], ['should'], ['i'], ['add'], ['to'], ['my'], ['rice'], ['crop'], ['?'], ['when'], ['to'], ['sow'], ['cotton'], ['?'], ['when'], ['to'], ['harvest'], ['wheat'], ['?'], ['how'], ['to'], ['enhanc'], ['yield'], ['of'], ['sugarcan'], ['?'], ['i'], ['want'], ['to'], ['grow'], ['paddi'], ['.'], ['how'], ['to'], ['grow'], ['maiz'], ['?'], ['which'], ['fertil'], ['can'], ['i'], ['use'], ['for'], ['sugarcan'], ['?'], ['how'], ['is'], ['the'], ['weather'], ['today'], ['?'], ['how'], ['is'], ['the'], ['weather'], ['tomorrow'], ['?'], ['what'], ['is'], ['the'], ['weather'], ['in'], ['